## Library

In [ ]:
import requests
import random
import pandas as pd
import time
import math
from copy import deepcopy
from statistics import mean

## Notebook

#### Requisição

In [ ]:
def request() -> list:   
    res = requests.get('http://localhost:3000/api/ep1')
    return res.json()

#### Tratamento de Dados

In [ ]:
def data_treatment(json:list):
    for i in range(len(json)):
        for key in json[i]:
            if ('prod' in key):
                if (type(json[i][key])!=dict):
                    qnt = json[i][key]
                    if (type(qnt)==str):
                        qnt = float(qnt)                  
                    if (qnt<0):
                        qnt = 0
                    if (key in [product for product in json[i] if ('prod' in product and (int(product[5]) < 9 and len(product) < 7))]):
                        json[i][key] = {'qnt':[round(qnt)], 'price':None}
                    else:
                        json[i][key] = {'qnt':[qnt], 'price':None}
        try:
            json[i]['date'] = time.ctime(json[i]['date'])
        except:
            continue
    return json

#### Gerar valores aleatórios

In [ ]:
def generate_random_price(n_min=1.0, n_max=100.0) -> float:
    return round(random.uniform(n_min, n_max), 2)

#### Precificação

##### Função logistica -> A mesma postada no doc pelo professor

In [ ]:
def logistic_function(v: float) -> float:
    return 0.5 + (1 / (1 + math.e ** -v))

##### Função para calcular a variação -> A mesma postada no doc pelo professor

In [ ]:
def calc_qnt_variation(last_week: list, current_week: list) -> float:
    return mean(current_week) - mean(last_week) / mean(last_week) if (mean(last_week) != 0) else 0

##### Função que calcula o preço que será utilizado na semana seguinte

In [ ]:
def calc_new_price(product_db: dict, sales_week: dict) -> dict:
    for product in product_db:
        if (product_db[product]['qnt'] != None):
            if (product not in sales_week.keys()):
                sales_week[product] = [0]
            product_db[product]['next_week_price'] = logistic_function(calc_qnt_variation(product_db[product]['qnt'], sales_week[product])) * product_db[product]['price']
        product_db[product]['qnt'] = sales_week[product]               
    return product_db

##### Função que atualiza o banco de dados (onde consta a quantidade de vendas da última semana e o preço dos produtos) e aplica o preço nos produtos do JSON

In [ ]:
def product_pricing(json: list, product_db: dict) -> tuple:
    sales_week = {}

    for transaction in json:
        for key in transaction:
            if ('prod' in key):
                if (key not in product_db.keys()):
                    transaction[key]['price'] = generate_random_price()
                    sales_week[key] = deepcopy(transaction[key]['qnt'])
                    product_db[key] = deepcopy(transaction[key])
                    product_db[key]['qnt'] = None
                    product_db[key]['next_week_price'] = product_db[key]['price']              
                else:
                    transaction[key]['price'] = deepcopy(product_db[key]['next_week_price'])
                    product_db[key]['price'] = product_db[key]['next_week_price']
                    if (key not in sales_week.keys()):
                        sales_week[key] = deepcopy(transaction[key]['qnt'])
                    else:
                        sales_week[key] += deepcopy(transaction[key]['qnt'])

    product_db = calc_new_price(product_db, sales_week)

    return json, product_db

#### Main

In [77]:
product_database = {}
weeks = 2

for week in range(weeks):
    json = data_treatment(request())
    json, product_database = product_pricing(json, product_database)

    json_db = pd.DataFrame(json)
    product_database_db = pd.DataFrame(product_database)

    print(f'\nSemana {week + 1}:\n')
    print('Json:')
    display(json_db)
    print('\nproduct_database:')
    display(product_database_db)


Semana 1:

Json:


,id,date,prod_1,prod_2,prod_3,prod_8,prod_10,prod_6,prod_7,prod_9,prod_0,prod_4,prod_5,prod_11
0,lagypxxm1nn4so5e8r6,Thu Jan 30 00:09:00 2020,"{'qnt': [7], 'price': 26.56}","{'qnt': [7], 'price': 48.32}","{'qnt': [8], 'price': 62.57}","{'qnt': [291], 'price': 35.92}","{'qnt': [587.926780637943], 'price': 97.17}",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,lagypxxmcy1lc2phpo8,Wed Jan 29 17:37:46 2020,NaN,NaN,NaN,"{'qnt': [512], 'price': 35.92}","{'qnt': [540.7003983100865], 'price': 97.17}","{'qnt': [22], 'price': 34.89}","{'qnt': [6], 'price': 82.33}","{'qnt': [631.5007774973963], 'price': 20.21}",NaN,NaN,NaN,NaN
2,lagypxxmxiq2pwc1cs9,Thu Jan 30 05:24:50 2020,"{'qnt': [11], 'price': 26.56}",NaN,"{'qnt': [46], 'price': 62.57}",NaN,NaN,"{'qnt': [44], 'price': 34.89}","{'qnt': [38], 'price': 82.33}","{'qnt': [48.72916429068284], 'price': 20.21}","{'qnt': [30], 'price': 87.56}",NaN,NaN,NaN
3,lagypxxmvgwezylx94,Wed Jan 29 18:25:37 2020,"{'qnt': [21], 'price': 26.56}",NaN,"{'qnt': [34], 'price': 62.57}","{'qnt': [11], 'price': 35.92}","{'qnt': [348.50057773459395], 'price': 97.17}","{'qnt': [26], 'price': 34.89}",NaN,NaN,NaN,"{'qnt': [27], 'price': 26.33}","{'qnt': [11], 'price': 1.74}","{'qnt': [89.28350478453667], 'price': 6.78}"
4,lagypxxmtbucv2vkzv,Wed Jan 29 17:18:07 2020,NaN,NaN,NaN,"{'qnt': [625], 'price': 35.92}","{'qnt': [448.3082554909861], 'price': 97.17}",NaN,NaN,"{'qnt': [12.905645120392364], 'price': 20.21}","{'qnt': [33], 'price': 87.56}",NaN,NaN,NaN
5,lagypxxm664tw1ebwkr,Thu Jan 30 05:05:18 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'qnt': [325.55048421166003], 'price': 20.21}","{'qnt': [25], 'price': 87.56}","{'qnt': [34], 'price': 26.33}",NaN,NaN
6,lagypxxnlx0brv4wfpi,Wed Jan 29 18:13:55 2020,NaN,NaN,"{'qnt': [43], 'price': 62.57}","{'qnt': [177], 'price': 35.92}","{'qnt': [654.3047658295377], 'price': 97.17}",NaN,NaN,"{'qnt': [40.59803603987257], 'price': 20.21}","{'qnt': [3], 'price': 87.56}","{'qnt': [40], 'price': 26.33}","{'qnt': [39], 'price': 1.74}","{'qnt': [231.88406825221327], 'price': 6.78}"
7,lagypxxnvbjtfy9usi,Thu Jan 30 07:25:43 2020,NaN,NaN,NaN,"{'qnt': [671], 'price': 35.92}","{'qnt': [481.80992931312255], 'price': 97.17}",NaN,NaN,"{'qnt': [15.151635350650706], 'price': 20.21}","{'qnt': [7], 'price': 87.56}","{'qnt': [17], 'price': 26.33}",NaN,NaN
8,lagypxxnrw0sveirqea,Thu Jan 30 02:24:15 2020,"{'qnt': [28], 'price': 26.56}","{'qnt': [33], 'price': 48.32}","{'qnt': [1], 'price': 62.57}","{'qnt': [454], 'price': 35.92}","{'qnt': [580.6170356893491], 'price': 97.17}","{'qnt': [12], 'price': 34.89}",NaN,"{'qnt': [24.351941778542685], 'price': 20.21}","{'qnt': [19], 'price': 87.56}",NaN,NaN,NaN
9,lagypxxns4kuap0qux,Wed Jan 29 19:40:32 2020,NaN,NaN,NaN,NaN,"{'qnt': [515.3532631433648], 'price': 97.17}","{'qnt': [40], 'price': 34.89}","{'qnt': [49], 'price': 82.33}","{'qnt': [267.7927487534772], 'price': 20.21}",NaN,"{'qnt': [13], 'price': 26.33}",NaN,NaN



product_database:


,prod_1,prod_2,prod_3,prod_8,prod_10,prod_6,prod_7,prod_9,prod_0,prod_4,prod_5,prod_11
qnt,"[7, 11, 21, 28, 8, 29, 6, 23, 33, 46, 4, 39, 4...","[7, 33, 7, 50, 20, 50, 23, 23, 2, 28, 17, 25]","[8, 46, 34, 43, 1, 5, 37, 42, 5, 42, 17, 27, 3...","[291, 512, 11, 625, 177, 671, 454, 746, 107, 2...","[587.926780637943, 540.7003983100865, 348.5005...","[22, 44, 26, 12, 40, 16, 38, 42, 42, 49, 35, 4...","[6, 38, 49, 35, 46, 3, 44, 48, 9, 3, 18, 4, 46...","[631.5007774973963, 48.72916429068284, 12.9056...","[30, 33, 25, 3, 7, 19, 14, 44, 7, 14, 25, 39, ...","[27, 34, 40, 17, 13, 12, 42, 4, 43, 8, 44, 48,...","[11, 39, 43, 15, 27, 3, 19, 48, 18, 35, 43, 19...","[89.28350478453667, 231.88406825221327, 164.96..."
price,26.56,48.32,62.57,35.92,97.17,34.89,82.33,20.21,87.56,26.33,1.74,6.78
next_week_price,26.56,48.32,62.57,35.92,97.17,34.89,82.33,20.21,87.56,26.33,1.74,6.78



Semana 2:

Json:


,id,date,prod_0,prod_1,prod_2,prod_4,prod_6,prod_7,prod_8,prod_11,prod_9,prod_10,prod_3,prod_5
0,lagypy5akq0gfueo2c,Wed Feb 5 17:35:34 2020,"{'qnt': [2], 'price': 87.56}","{'qnt': [37], 'price': 26.56}","{'qnt': [9], 'price': 48.32}","{'qnt': [36], 'price': 26.33}","{'qnt': [12], 'price': 34.89}","{'qnt': [50], 'price': 82.33}","{'qnt': [236], 'price': 35.92}","{'qnt': [199.94745084079813], 'price': 6.78}",NaN,NaN,NaN,NaN
1,lagypy5ankn8lbd82gp,Wed Feb 5 19:46:17 2020,NaN,"{'qnt': [2], 'price': 26.56}",NaN,"{'qnt': [21], 'price': 26.33}",NaN,"{'qnt': [32], 'price': 82.33}",NaN,"{'qnt': [0], 'price': 6.78}","{'qnt': [0], 'price': 20.21}","{'qnt': [0], 'price': 97.17}",NaN,NaN
2,lagypy5ausrjtrkncui,Wed Feb 5 19:18:41 2020,NaN,NaN,"{'qnt': [19], 'price': 48.32}",NaN,NaN,NaN,"{'qnt': [5], 'price': 35.92}","{'qnt': [357.89026407180717], 'price': 6.78}","{'qnt': [52.70620067841298], 'price': 20.21}","{'qnt': [224.14303079720474], 'price': 97.17}","{'qnt': [47], 'price': 62.57}","{'qnt': [44], 'price': 1.74}"
3,lagypy5a42iyzhl0xue,Thu Feb 6 00:19:06 2020,"{'qnt': [4], 'price': 87.56}","{'qnt': [27], 'price': 26.56}",NaN,"{'qnt': [42], 'price': 26.33}",NaN,NaN,NaN,NaN,NaN,"{'qnt': [338.20930152526546], 'price': 97.17}",NaN,"{'qnt': [13], 'price': 1.74}"
4,lagypy5awebhc864chh,Wed Feb 5 21:25:14 2020,NaN,NaN,"{'qnt': [33], 'price': 48.32}",NaN,"{'qnt': [9], 'price': 34.89}",NaN,"{'qnt': [0], 'price': 35.92}","{'qnt': [0], 'price': 6.78}","{'qnt': [0], 'price': 20.21}",NaN,"{'qnt': [42], 'price': 62.57}",NaN
5,lagypy5ao3jg4hw8k4r,Thu Feb 6 07:00:37 2020,NaN,NaN,NaN,"{'qnt': [15], 'price': 26.33}",NaN,NaN,NaN,NaN,NaN,"{'qnt': [0], 'price': 97.17}",NaN,NaN
6,lagypy5a9jlxbhpm2yj,Wed Feb 5 21:35:11 2020,NaN,NaN,"{'qnt': [21], 'price': 48.32}",NaN,"{'qnt': [37], 'price': 34.89}","{'qnt': [16], 'price': 82.33}",NaN,NaN,"{'qnt': [157.63924674321214], 'price': 20.21}",NaN,NaN,"{'qnt': [37], 'price': 1.74}"
7,lagypy5asewjnet654,Thu Feb 6 02:28:39 2020,"{'qnt': [45], 'price': 87.56}","{'qnt': [14], 'price': 26.56}","{'qnt': [14], 'price': 48.32}","{'qnt': [33], 'price': 26.33}",NaN,NaN,NaN,"{'qnt': [312.3013804600857], 'price': 6.78}","{'qnt': [37.49740050049645], 'price': 20.21}","{'qnt': [371.68725917288253], 'price': 97.17}",NaN,"{'qnt': [45], 'price': 1.74}"
8,lagypy5ammvebvjm1h,Thu Feb 6 00:24:49 2020,NaN,NaN,"{'qnt': [34], 'price': 48.32}","{'qnt': [40], 'price': 26.33}",NaN,"{'qnt': [18], 'price': 82.33}","{'qnt': [0], 'price': 35.92}",NaN,"{'qnt': [0], 'price': 20.21}",NaN,NaN,NaN
9,lagypy5a68n9gtnzews,Thu Feb 6 01:01:42 2020,NaN,"{'qnt': [8], 'price': 26.56}",NaN,NaN,"{'qnt': [39], 'price': 34.89}","{'qnt': [37], 'price': 82.33}",NaN,NaN,"{'qnt': [49.37793689481849], 'price': 20.21}","{'qnt': [405.3896523914009], 'price': 97.17}","{'qnt': [17], 'price': 62.57}",NaN



product_database:


,prod_1,prod_2,prod_3,prod_8,prod_10,prod_6,prod_7,prod_9,prod_0,prod_4,prod_5,prod_11
qnt,"[37, 2, 27, 14, 8, 23, 26, 8, 21, 10, 8, 21, 2...","[9, 19, 33, 21, 14, 34, 24, 12, 21, 19, 24, 33...","[47, 42, 17, 2, 43, 12, 36, 11, 39, 48, 14, 2]","[236, 5, 0, 0, 31, 270, 19, 40, 0, 21, 0, 286,...","[0, 224.14303079720474, 338.20930152526546, 0,...","[12, 9, 37, 39, 41, 44, 40, 41, 3, 36, 1]","[50, 32, 16, 18, 37, 24, 32, 20, 37, 40, 42, 2...","[0, 52.70620067841298, 0, 157.63924674321214, ...","[2, 4, 45, 2, 11, 41, 36, 20, 43, 47]","[36, 21, 42, 15, 33, 40, 40, 25, 38, 43, 17, 2...","[44, 13, 37, 45, 46, 16, 24, 19, 25, 45, 10, 1...","[199.94745084079813, 0, 357.89026407180717, 0,..."
price,26.56,48.32,62.57,35.92,97.17,34.89,82.33,20.21,87.56,26.33,1.74,6.78
next_week_price,39.839999,72.48,93.855,53.88,145.755,52.335,123.495,30.315,131.34,39.495,2.61,10.17


In [93]:

df = product_database_db.copy()

somas = []
prices = []
colunas = list(df.columns)

for i in range(df.shape[1]):
   somas.append(sum(df.iloc[0,i]))
   prices.append(df.iloc[1,i])
   

lista_de_tuplas = list(zip(colunas, somas, prices))

new_df = pd.DataFrame(lista_de_tuplas, columns = ['produto', 'soma', 'preço'])

,produto,soma,preço
0,prod_1,272.000000,26.56
1,prod_2,282.000000,48.32
2,prod_3,313.000000,62.57
3,prod_8,938.000000,35.92
4,prod_10,2415.793238,97.17
5,prod_6,303.000000,34.89
6,prod_7,377.000000,82.33
7,prod_9,2195.210737,20.21
8,prod_0,251.000000,87.56
9,prod_4,444.000000,26.33


In [115]:
#new_df.iloc[linha, coluna]
#new_df.iloc[0,1]
# receita = [new_df.iloc[linha,coluna]*
new_df['receita'] = new_df['soma'] * new_df['preço']
new_df

,produto,soma,preço,receita
0,prod_1,272.000000,26.56,7224.320000
1,prod_2,282.000000,48.32,13626.240000
2,prod_3,313.000000,62.57,19584.410000
3,prod_8,938.000000,35.92,33692.960000
4,prod_10,2415.793238,97.17,234742.628929
5,prod_6,303.000000,34.89,10571.670000
6,prod_7,377.000000,82.33,31038.410000
7,prod_9,2195.210737,20.21,44365.208988
8,prod_0,251.000000,87.56,21977.560000
9,prod_4,444.000000,26.33,11690.520000


In [117]:
receita = sum(new_df['receita'])
receita

445439.77148048364